In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
from paths import PF_DB_PATH
from IPython.display import Markdown as md
from datetime import datetime

In [65]:
def display_list(lst: list, sep=", ") -> str:
    # join list items to return a merged str for display
    return sep.join(lst)

# Raw Data
Dataset was downloaded as separated `.xlsx` files, which are then merged into a single SQLite database. The SQL database consists of 2 tables:

In [66]:
con= sqlite3.connect(PF_DB_PATH)
raw_records= pd.read_sql_query("SELECT * FROM record", con)
raw_property=  pd.read_sql_query("SELECT * FROM property", con)

In [67]:
md(
    f"""
1. records\n
    This table contains {raw_records.shape[0]:,} property sales records, each record contains the following data:
    * `id`: unique identification of the sales record, formed by merging address, postcode and sale date
    * `record_date`: date of sale (from {raw_records.record_date.min()} to {raw_records.record_date.max()})
    * `price`: final sale price
    * `sale_type`: type of sale ({display_list(raw_records.sale_type.unique().tolist())})
    * `prop_id`: id of the property, formed by merging address and postcode
    * `property_type`: type of property ({display_list(raw_records.property_type.unique().tolist())})
    * `bedrooms`: number of bedrooms
    * `bathrooms`: number of bathrooms
    * `carparks`: number of car parks
2. property\n
    This table contains {raw_property.shape[0]:,} properties that have associated sales record(s), each property details consist of the following data:
    * `address`: address with postcode attached, act as unique identification
    * `street`: street name
    * `locality`: suburb name ({raw_property.locality.nunique()} in total)
    * `postcode`: postcode ({raw_property.postcode.nunique()} in total)
    * `land_area`: land area
"""
)


1. records

    This table contains 731,552 property sales records, each record contains the following data:
    * `id`: unique identification of the sales record, formed by merging address, postcode and sale date
    * `record_date`: date of sale (from 2000-01-05 to 2022-01-04)
    * `price`: final sale price
    * `sale_type`: type of sale (Agents Advice - Sale, Normal Sale, Multi-Sale, Part Sale, Unknown)
    * `prop_id`: id of the property, formed by merging address and postcode
    * `property_type`: type of property (Commercial, Unit, Other, House, Vacant Land, Industrial)
    * `bedrooms`: number of bedrooms
    * `bathrooms`: number of bathrooms
    * `carparks`: number of car parks
2. property

    This table contains 455,865 properties that have associated sales record(s), each property details consist of the following data:
    * `address`: address with postcode attached, act as unique identification
    * `street`: street name
    * `locality`: suburb name (844 in total)
    * `postcode`: postcode (312 in total)
    * `land_area`: land area


In [68]:
raw_records.describe(include = 'all')

,id,record_date,price,sale_type,prop_id,property_type,bedrooms,bathrooms,carparks
count,731552,731552,7.315520e+05,731552,731552,731552,731552.000000,731552.000000,731552.000000
unique,731552,8020,NaN,5,455865,6,NaN,NaN,NaN
top,"SUITE 311/368 SUSSEX ST, 2000_2021-12-16 00:00:00",2021-06-05,NaN,Normal Sale,"81 COURALLIE AVE, 2140",House,NaN,NaN,NaN
freq,1,457,NaN,712227,250,349058,NaN,NaN,NaN
mean,NaN,NaN,8.016123e+05,NaN,NaN,NaN,2.941512,1.758117,1.502566
std,NaN,NaN,2.019461e+06,NaN,NaN,NaN,54.283706,54.234449,2.211600
min,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,3.420000e+05,NaN,NaN,NaN,2.000000,1.000000,1.000000
50%,NaN,NaN,5.600000e+05,NaN,NaN,NaN,3.000000,2.000000,1.000000
75%,NaN,NaN,8.910000e+05,NaN,NaN,NaN,4.000000,2.000000,2.000000


In [69]:
raw_property.describe(include='all')

,address,street,locality,postcode,land_area
count,455865,455865,455865,455865.000000,4.232550e+05
unique,455865,21958,844,NaN,NaN
top,"SUITE 311/368 SUSSEX ST, 2000",PACIFIC HWY,MOSMAN,NaN,NaN
freq,1,3755,2814,NaN,NaN
mean,NaN,NaN,NaN,2152.094122,3.828190e+03
std,NaN,NaN,NaN,79.614767,4.209191e+05
min,NaN,NaN,NaN,2000.000000,0.000000e+00
25%,NaN,NaN,NaN,2089.000000,4.740000e+02
50%,NaN,NaN,NaN,2147.000000,6.890000e+02
75%,NaN,NaN,NaN,2216.000000,1.170000e+03


# Data Cleaning

In [71]:
raw_records.isna().sum()

id               0
record_date      0
price            0
sale_type        0
prop_id          0
property_type    0
bedrooms         0
bathrooms        0
carparks         0
dtype: int64

In [72]:
raw_property.isna().sum()

address          0
street           0
locality         0
postcode         0
land_area    32610
dtype: int64

In [84]:
# raw dataframe
df_raw= (
    pd.merge(raw_records, raw_property, left_on='prop_id', right_on='address', how= 'left')
    .drop(columns=['prop_id'])
)

# filtered dataframe
last_record_date = '2021-12-31'
df= df_raw.query(f'bedrooms<=5 & bathrooms<=5 & carparks <=3')
df= df[df.record_date <= last_record_date]
df['year']= pd.DatetimeIndex(df.record_date).year
df.describe(include='all')


,id,record_date,price,sale_type,property_type,bedrooms,bathrooms,carparks,address,street,locality,postcode,land_area,year
count,697935,697935,6.979350e+05,697935,697935,697935.000000,697935.000000,697935.000000,697935,697935,697935,697935.000000,6.433930e+05,697935.000000
unique,697935,8016,NaN,5,6,NaN,NaN,NaN,435923,21687,834,NaN,NaN,NaN
top,"SUITE 311/368 SUSSEX ST, 2000_2021-12-16 00:00:00",2021-06-05,NaN,Normal Sale,Unit,NaN,NaN,NaN,"81 COURALLIE AVE, 2140",PACIFIC HWY,CAMMERAY,NaN,NaN,NaN
freq,1,433,NaN,680061,338648,NaN,NaN,NaN,250,6551,4331,NaN,NaN,NaN
mean,NaN,NaN,7.674810e+05,NaN,NaN,2.749742,1.608812,1.361115,NaN,NaN,NaN,2150.289793,5.236439e+03,2011.536799
std,NaN,NaN,1.906581e+06,NaN,NaN,1.045647,0.705885,0.680382,NaN,NaN,NaN,80.458172,6.355019e+05,6.451065
min,NaN,NaN,0.000000e+00,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,2000.000000,0.000000e+00,2000.000000
25%,NaN,NaN,3.400000e+05,NaN,NaN,2.000000,1.000000,1.000000,NaN,NaN,NaN,2088.000000,4.680000e+02,2006.000000
50%,NaN,NaN,5.500000e+05,NaN,NaN,3.000000,2.000000,1.000000,NaN,NaN,NaN,2145.000000,6.960000e+02,2013.000000
75%,NaN,NaN,8.650000e+05,NaN,NaN,3.000000,2.000000,2.000000,NaN,NaN,NaN,2214.000000,1.348000e+03,2017.000000


In [91]:
md(f'''
There is no missing values in both tables. For records, unusually high number of bedrooms, bathrooms and car parks were found, possible due to the sale of buildings as a whole. For normal household, only properties with maximum 5 bedrooms/bathrooms and 3 car parks were considered.

Also, to aid yearly aggregation, a `year` column was generated from `record_date`. It should be noted that only sales records on or before {last_record_date} were considered to ensure accuracy of annual statistics.

Consequently, a combined dataset was then formed by merging the two tables with left join on `records`. The final dataset for analysis consists of {df.shape[0]:,} records ({df.shape[0]/df_raw.shape[0]:.0%} of available records).
''')


There is no missing values in both tables. For records, unusually high number of bedrooms, bathrooms and car parks were found, possible due to the sale of buildings as a whole. For normal household, only properties with maximum 5 bedrooms/bathrooms and 3 car parks were considered.

Also, to aid yearly aggregation, a `year` column was generated from `record_date`. It should be noted that only sales records on or before 2021-12-31 were considered to ensure accuracy of annual statistics.

Consequently, a combined dataset was then formed by merging the two tables with left join on `records`. The final dataset for analysis consists of 697,935 records (95% of available records).
